In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import overpy
import polyline
import overpass

In [3]:
orig_df = pd.read_csv("DOT_Traffic_Speeds_NBE.csv")
orig_df.head()

/tmp/ipykernel_23070/4030752796.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  orig_df = pd.read_csv("DOT_Traffic_Speeds_NBE.csv")


,ID,SPEED,TRAVEL_TIME,STATUS,DATA_AS_OF,LINK_ID,LINK_POINTS,ENCODED_POLY_LINE,ENCODED_POLY_LINE_LVLS,OWNER,TRANSCOM_ID,BOROUGH,LINK_NAME
0,441,56.54,139.0,0.0,08/22/2021 03:49:04 PM,4616201.0,"40.56058,-74.199581 40.568141,-74.192011 40.57...",s~pvFjb{cMgn@in@cK{F}JwBwIUyGp@qFv@qG\\\\\\|@w...,BBBBBBBBB,NYC_DOT_LIC,4616201.0,Staten Island,WSE S VICTORY BOULEVARD - ARDEN AVENUE
1,436,58.40,89.0,0.0,08/22/2021 03:49:04 PM,4616202.0,"40.56058,-74.199581 40.55926,-74.20113 40.5584...",s~pvFjb{cMfGtHxCrFvB~FdBhGrFz\t@`DpC~JpCnHpBhE...,BBBBBBBBBBBBBB,NYC_DOT_LIC,4616202.0,Staten Island,WSE S ARDEN AVENUE - BLOOMINGDALE ROAD
2,418,41.63,51.0,0.0,08/22/2021 03:49:04 PM,4616205.0,"40.60425,-74.052321 40.6038804,-74.053491 40.6...",qoyvF~i~bMhAhF`ArEdAtEbDtNp@lDRvDKrDc@~Dm@bFy@...,BBBBBBBBBBBBB,NYC_DOT_LIC,4616205.0,Staten Island,VNB W-SIE W SI GANTRY LOWER LEVEL - FINGERBOAR...
3,385,45.98,31.0,0.0,08/22/2021 03:49:04 PM,4616208.0,"40.6077805,-74.14091 40.60826,-74.132101",sezvFtsocM_Bav@,BB,NYC_DOT_LIC,4616208.0,Staten Island,SIE W BRADLEY AVENUE - WOOLEY AVENUE
4,390,57.16,69.0,0.0,08/22/2021 03:49:04 PM,4616209.0,"40.6152105,-74.157401 40.61231,-74.15362 40.60...",at{vFvzrcMbQsVlP{UjCkFbBsFxAkJ\{HKiS,BBBBBBBB,NYC_DOT_LIC,4616209.0,Staten Island,SIE W WOOLEY AVENUE - RICHMOND AVENUE


In [4]:
print(rf'len of original df -- {len(orig_df)}')

len of original df -- 2815917


In [5]:
df = orig_df
# convert date column to date-time format
df['as_of'] = pd.to_datetime(df['DATA_AS_OF'])

# filter for one week of data
start_date = '2021-08-22' # replace with your desired start date
end_date = '2021-08-30' # replace with your desired end date

mask = (df['as_of'] >= start_date) & (df['as_of'] <= end_date)
one_week_df = df.loc[mask]

# print the filtered DataFrame
len(one_week_df)

254956

In [6]:
# REMOVE COLUMNS "STATUS, TRANSCOM_ID, ENCODED_POLY_LINE_LVLS, OWNER
df_trim = one_week_df.drop(['STATUS','TRANSCOM_ID', 'ENCODED_POLY_LINE_LVLS', 'OWNER', 'DATA_AS_OF', 'as_of'], axis=1)
df_trim.head()

,ID,SPEED,TRAVEL_TIME,LINK_ID,LINK_POINTS,ENCODED_POLY_LINE,BOROUGH,LINK_NAME
0,441,56.54,139.0,4616201.0,"40.56058,-74.199581 40.568141,-74.192011 40.57...",s~pvFjb{cMgn@in@cK{F}JwBwIUyGp@qFv@qG\\\\\\|@w...,Staten Island,WSE S VICTORY BOULEVARD - ARDEN AVENUE
1,436,58.40,89.0,4616202.0,"40.56058,-74.199581 40.55926,-74.20113 40.5584...",s~pvFjb{cMfGtHxCrFvB~FdBhGrFz\t@`DpC~JpCnHpBhE...,Staten Island,WSE S ARDEN AVENUE - BLOOMINGDALE ROAD
2,418,41.63,51.0,4616205.0,"40.60425,-74.052321 40.6038804,-74.053491 40.6...",qoyvF~i~bMhAhF`ArEdAtEbDtNp@lDRvDKrDc@~Dm@bFy@...,Staten Island,VNB W-SIE W SI GANTRY LOWER LEVEL - FINGERBOAR...
3,385,45.98,31.0,4616208.0,"40.6077805,-74.14091 40.60826,-74.132101",sezvFtsocM_Bav@,Staten Island,SIE W BRADLEY AVENUE - WOOLEY AVENUE
4,390,57.16,69.0,4616209.0,"40.6152105,-74.157401 40.61231,-74.15362 40.60...",at{vFvzrcMbQsVlP{UjCkFbBsFxAkJ\{HKiS,Staten Island,SIE W WOOLEY AVENUE - RICHMOND AVENUE


In [7]:
polylist = list(df_trim.ENCODED_POLY_LINE.unique())


def get_num_lanes(encoded_polyline):
    
    try:
        decoded_polyline = polyline.decode(encoded_polyline)
    
    
        api = overpy.Overpass()
        num_lanes = []
        for lat, lon in decoded_polyline:
            result = api.query(f"""
                way(around:1,{lat},{lon})["highway"];
                (._;>;);
                out body;
            """)
            for way in result.ways:
                num_lanes.append(way.tags.get('lanes', 'unknown'))
        return num_lanes
    except:
        return None

max_lanes_dict = {}
for encoded_polyline in polylist:
    num_lanes = get_num_lanes(encoded_polyline)
    if num_lanes:
        max_lanes = max(num_lanes)
        max_lanes_dict[encoded_polyline] = max_lanes

In [ ]:
max_lanes_dict

In [ ]:
len(df_trim.LINK_NAME.unique())